In [ ]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, cohen_kappa_score



In [ ]:
class FeatureCORAL(nn.Module):
    def __init__(self, input_dim, num_classes=8):
        super(FeatureCORAL, self).__init__()
        # 1. Shared Weight Vector (w): Projects features to a single "Proficiency Score"
        self.linear = nn.Linear(input_dim, 1, bias=False)

        # 2. Independent Biases (b_k): The cut-points between levels
        # We need K-1 biases for K classes
        self.biases = nn.Parameter(torch.zeros(num_classes - 1))

    def forward(self, x):
        # x shape: [batch_size, input_dim]
        # score shape: [batch_size, 1]
        score = self.linear(x)

        # Adding biases: [batch_size, 1] + [K-1] -> [batch_size, K-1] (Broadcasting)
        logits = score + self.biases
        return logits

def task_encoding(y, num_classes=8):
    batch_size = y.size(0)
    # Create a matrix of thresholds [0, 1, ..., K-2]
    thresholds = torch.arange(num_classes - 1).expand(batch_size, num_classes - 1).to(y.device)
    # Broadcast y: [batch_size, 1]
    y_expanded = y.view(-1, 1).expand(batch_size, num_classes - 1)

    # Return 1 if label > threshold
    return (y_expanded > thresholds).float()

def predict_rank(logits):
    """Sum probabilities to get expected rank (Proba -> Sigmoid -> Sum -> Round)"""
    probs = torch.sigmoid(logits)
    pred_rank = torch.sum(probs, dim=1)
    return torch.round(pred_rank).detach().cpu().numpy().astype(int)



In [ ]:

df = pd.read_csv("training_features_extended_final.csv")  # Make sure this filename is correct
target_col = 'label' if 'label' in df.columns else 'labels'

X = df.drop(columns=[target_col]).values.astype(np.float32)
y = df[target_col].values.astype(np.int64)
num_classes = 8
input_dim = X.shape[1]



In [ ]:

skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

results = {'MAE': [], 'RMSE': [], 'QWK': []}
fold = 1

print(f"Starting Feature-CORAL (PyTorch) on {input_dim} features...")
print("-" * 60)

for train_index, test_index in skf.split(X, y):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]

    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)

    X_train_t = torch.tensor(X_train)
    y_train_t = torch.tensor(y_train)
    X_test_t = torch.tensor(X_test)

    model = FeatureCORAL(input_dim, num_classes)
    optimizer = optim.Adam(model.parameters(), lr=0.01, weight_decay=1e-4)
    loss_fn = nn.BCEWithLogitsLoss() # Binary Cross Entropy


    model.train()
    epochs = 1000  # Features are simple, converges fast
    for epoch in range(epochs):
        optimizer.zero_grad()

        logits = model(X_train_t)

        # Create CORAL targets: [1,1,0,0...]
        targets = task_encoding(y_train_t, num_classes)

        loss = loss_fn(logits, targets)
        loss.backward()
        optimizer.step()


    model.eval()
    with torch.no_grad():
        test_logits = model(X_test_t)
        y_pred = predict_rank(test_logits)


    mae = mean_absolute_error(y_test, y_pred)
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    qwk = cohen_kappa_score(y_test, y_pred, weights='quadratic')

    results['MAE'].append(mae)
    results['RMSE'].append(rmse)
    results['QWK'].append(qwk)

    print(f"Fold {fold}: QWK = {qwk:.4f} | MAE = {mae:.4f} | RMSE = {rmse:.4f}")
    fold += 1


print("-" * 60)
print("FINAL RESULTS (Feature-CORAL):")
for metric, values in results.items():
    print(f"{metric}: {np.mean(values):.4f} ± {np.std(values):.4f}")

Starting Feature-CORAL (PyTorch) on 24 features...
------------------------------------------------------------
Fold 1: QWK = 0.7593 | MAE = 0.6184 | RMSE = 0.9711
Fold 2: QWK = 0.7419 | MAE = 0.7105 | RMSE = 1.0044
Fold 3: QWK = 0.7562 | MAE = 0.6623 | RMSE = 0.9665
Fold 4: QWK = 0.7676 | MAE = 0.6564 | RMSE = 0.9457
Fold 5: QWK = 0.7703 | MAE = 0.6916 | RMSE = 0.9549
------------------------------------------------------------
FINAL RESULTS (Feature-CORAL):
MAE: 0.6678 ± 0.0316
RMSE: 0.9685 ± 0.0200
QWK: 0.7591 ± 0.0100
